In [27]:
import pandas as pd
import numpy as np

Prescriber Data  
https://data.cms.gov/Medicare-Part-D/Medicare-Provider-Utilization-and-Payment-Data-201/77gb-8z53  
Physician Compare data  
https://data.medicare.gov/Physician-Compare/Physician-Compare-National-Downloadable-File/mj5m-pzi6

### ** Consider reodering cells below to query antipsychotics before merging


#### read in, select columns, rename columns for Part D Prescriber data and Physician Data

In [28]:
%time
prescriber_df = pd.read_csv('../data/archive/Medicare_Provider_Utilization_and_Payment_Data__2017_Part_D_Prescriber.csv', 
                            usecols=['npi','nppes_provider_last_org_name','nppes_provider_first_name','nppes_provider_city',
                                   'nppes_provider_state','specialty_description','generic_name','bene_count_ge65',
                                   'total_claim_count_ge65'])
prescriber_df = prescriber_df.rename(columns={'nppes_provider_last_org_name':'last_name','nppes_provider_first_name':'first_name',
                                              'nppes_provider_city':'city','nppes_provider_state': 'state',
                                              'specialty_description':'specialty','generic_name':'drug',})

Wall time: 0 ns


In [29]:
list_of_all_antipsychs = ['ARIPIPRAZOLE',
'ARIPIPRAZOLE LAUROXIL',
'CHLORPROMAZINE HCL',
'CLOZAPINE',
'ILOPERIDONE',
'FLUPHENAZINE DECANOATE',
'FLUPHENAZINE HCL',
'ZIPRASIDONE HCL',
'ZIPRASIDONE MESYLATE',
'HALOPERIDOL LACTATE',
'HALOPERIDOL DECANOATE',
'HALOPERIDOL',
'PALIPERIDONE',
'PALIPERIDONE PALMITATE',
'LURASIDONE HCL',
'LOXAPINE SUCCINATE',
'MOLINDONE HCL',
'PIMAVANSERIN TARTRATE',
'OLANZAPINE',
'OLANZAPINE/FLUOXETINE HCL',
'PIMOZIDE',
'PERPHENAZINE',
'PERPHENAZINE/AMITRIPTYLINE HCL',
'QUETIAPINE FUMARATE',
'BREXPIPRAZOLE',
'RISPERIDONE',
'RISPERIDONE MICROSPHERES',
'ASENAPINE MALEATE',
'THIORIDAZINE HCL',
'THIOTHIXENE',
'TRIFLUOPERAZINE HCL',
'CARIPRAZINE HCL',
'OLANZAPINE PAMOATE']

In [4]:
prescriber_df.shape

(25209130, 9)

In [30]:
prescriber_df = prescriber_df.query('drug in @list_of_all_antipsychs')

In [6]:
prescriber_df.shape

(509235, 9)

In [7]:
prescriber_df[prescriber_df['state']=='' .isna().sum()

npi                            0
last_name                      2
first_name                     3
city                           0
state                          0
specialty                      0
drug                           0
bene_count_ge65           356420
total_claim_count_ge65    161505
dtype: int64

In [8]:
prescriber_df.notnull().sum()

npi                       509235
last_name                 509233
first_name                509232
city                      509235
state                     509235
specialty                 509235
drug                      509235
bene_count_ge65           152815
total_claim_count_ge65    347730
dtype: int64

In [9]:
prescriber_df.nunique()

npi                       149764
last_name                  62083
first_name                 20671
city                        8361
state                         59
specialty                    118
drug                          33
bene_count_ge65              161
total_claim_count_ge65       755
dtype: int64

In [15]:
total_benef = prescriber_df.groupby(['state'])['bene_count_ge65'].sum().to_frame().sort_values(by= ['bene_count_ge65'],ascending=False)

In [20]:
ak_df = prescriber_df.loc[prescriber_df['state']=='AK']

In [22]:
ak_df.bene_count_ge65.sum()

17.0

In [26]:
ak_df.npi.count()

903

#### Bring in physician data and merge

In [3]:
physician_df = pd.read_csv('../data/archive/Physician_Compare_National_Downloadable_File.csv',
                           usecols=['NPI','Hospital affiliation CCN 1','Hospital affiliation LBN 1','Hospital affiliation CCN 2',
                                   'Hospital affiliation LBN 2','Hospital affiliation CCN 3','Hospital affiliation LBN 3',
                                   'Hospital affiliation CCN 4','Hospital affiliation LBN 4','Hospital affiliation CCN 5',
                                   'Hospital affiliation LBN 5'])
physician_df = physician_df.rename(columns={'NPI': 'npi2','Hospital affiliation CCN 1':'ccn1','Hospital affiliation LBN 1':'lbn1',
                                            'Hospital affiliation CCN 2':'ccn2','Hospital affiliation LBN 2':'lbn2',
                                            'Hospital affiliation CCN 3':'ccn3','Hospital affiliation LBN 3':'lbn3',
                                            'Hospital affiliation CCN 4':'ccn4','Hospital affiliation LBN 4':'lbn4',
                                            'Hospital affiliation CCN 5':'ccn5','Hospital affiliation LBN 5':'lbn5'})

C:\Users\ben\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (29,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#prescriber_and_physician_df = pd.merge(prescriber_df,physician_df,left_on='npi',right_on='npi2')
#prescriber_and_physician_df.head(1)

In [10]:
prescriber_and_physician_df.shape

(2183992, 11)

In [13]:
prescriber_and_physician_df = prescriber_and_physician_df.drop_duplicates()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ben\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-e1aec52f0918>", line 1, in <module>
    all_drugs_usa_over65_drop_dup = all_drugs_usa_over65.drop_duplicates()
  File "C:\Users\ben\anaconda3\lib\site-packages\pandas\core\frame.py", line 4811, in drop_duplicates
    duplicated = self.duplicated(subset, keep=keep)
  File "C:\Users\ben\anaconda3\lib\site-packages\pandas\core\frame.py", line 4888, in duplicated
    labels, shape = map(list, zip(*map(f, vals)))
  File "C:\Users\ben\anaconda3\lib\site-packages\pandas\core\frame.py", line 4863, in f
    vals, size_hint=min(len(self), _SIZE_HINT_LIMIT)
  File "C:\Users\ben\anaconda3\lib\site-packages\pandas\core\algorithms.py", line 636, in factorize
    values, na_sentinel=na_sentinel, size_hint=size_hint, na_value=na_value
  File "C:\Users\ben\anaconda3\lib\site-packages\

KeyboardInterrupt: 

In [15]:
prescriber_and_physician_df.shape

(20457591, 20)

In [18]:
prescriber_and_physician_df.head(2)

,npi,last_name,first_name,city,state,specialty,drug,bene_count_ge65,total_claim_count_ge65,npi2,ccn1,lbn2,ccn2,lbn2,ccn3,lbn3,ccn4,lbn4,ccn5,lbn5
344,1952306417,PARKHURST,REBECCA,ERIE,PA,Physician Assistant,QUETIAPINE FUMARATE,NaN,43.0,1952306417,390146.0,WARREN GENERAL HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,1427075894,CHOUNZOM,TENZING,MONTEAGLE,TN,Internal Medicine,RISPERIDONE,NaN,25.0,1427075894,440058.0,SOUTHERN TENNESSEE REGIONAL HLTH SYSTEM WINCHE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
prescriber_and_physician_df.shape

(380112, 20)

In [21]:
prescriber_and_physician_df.to_csv('../data/antipsych_drugs_usa_over65.csv', index = False)